# Généricité

## Syntaxe des bornes de trait

En Rust, la définition de types peut être paramétrée avec des paramètres de type.
Nous avons déjà vu quelques exemples: `Option<T>` et `Result<T,E>`.
Ces deux types en particulier n'imposent pas de contraintes sur leurs paramètres `T` et `E`, mais il est possible de le faire.

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

struct Couple<T: Lifeform> {
    male: T,
    female: T,
}

Pour une fonction générique :

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

fn remove_dead_classic<T: Lifeform>(things: &mut Vec<T>) {
    things.retain(|thing| thing.is_alive())
}

fn remove_dead_impl(things: &mut Vec<impl Lifeform>) {
//                                   ^^^^^^^^^^^^^
//                         sucre syntaxique pour la généricité
    things.retain(|thing| thing.is_alive())
}

Plusieurs contraintes peuvent être spécifiées, pour indiquer qu'un paramètre de type doit implémenter plusieurs traits :

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

fn remove_dead<T: Lifeform + Copy>(things: &mut Vec<T>) {
    things.retain(|thing| thing.is_alive())
}

Avec la programmation générique, Rust utilise du dispatch statique avec la mono-morphisation :

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

#[derive(Debug)]
struct Kelp { size: i32 }

impl Lifeform for Kelp {
    fn is_alive(&self) -> bool { self.size <= 0 }
}

#[derive(Debug)]
struct Fish { age: usize }

impl Lifeform for Fish {
    fn is_alive(&self) -> bool { self.age >= 20 }
}

fn remove_dead<T: Lifeform>(things: &mut Vec<T>) {
    things.retain(|thing| thing.is_alive())
}

fn main() {
    let mut kelps = vec![Kelp{size: 10}, Kelp{size: 0}];
    let mut fishes = vec![Fish{age: 21}, Fish{age: 3}];
    
    remove_dead(&mut kelps);
    //          ^^^^^^^^^^ &mut Vec<Kelp>
    remove_dead(&mut fishes);
    //          ^^^^^^^^^^^ &mut Vec<Fish>
    
    println!("kelps = {kelps:?}");
    println!("fishes = {fishes:?}");
}
main();

## Dispatch dynamique (trait objects)

Rust supporte aussi le disptach dynamique avec la syntaxe `dyn Trait`.

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
    fn eat(&mut self);
}

#[derive(Debug)]
struct Kelp { size: i32 }

impl Lifeform for Kelp {
    fn is_alive(&self) -> bool { self.size <= 0 }
    fn eat(&mut self) { self.size += 1; }
}

#[derive(Debug)]
struct Fish { age: usize }

impl Lifeform for Fish {
    fn is_alive(&self) -> bool { self.age >= 20 }
    fn eat(&mut self) { self.age += 1; }
}

fn simulate(thing: &mut dyn Lifeform) {
    if thing.is_alive() {
        thing.eat();
    }
}

fn main() {
    let mut kelp = Kelp{size: 10};
    let mut fish = Fish{age: 21};
    
    simulate(&mut kelp);
    //       ^^^^^^^^^ &mut Kelp
    simulate(&mut fish);
    //       ^^^^^^^^^ &mut Fish
    
    println!("{kelp:?}");
    println!("{fish:?}");
    println!("sizeof &mut Kelp = {}", std::mem::size_of::<&mut Kelp>());
    println!("sizeof &mut dyn Lifeform = {}", std::mem::size_of::<&mut dyn Lifeform>());
}
main();

## `impl` en position de retour

La forme `impl Trait` peut aussi être utilisé pour le type de retour de la fonction pour spécifier que le type implémente un trait, mais sans spécifier exactement lequel.

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

#[derive(Debug)]
struct Kelp { size: i32 }

impl Lifeform for Kelp {
    fn is_alive(&self) -> bool { self.size <= 0 }
}

fn return_lifeform() -> impl Lifeform { // <-- ici
    Kelp { size: 10 }
}

En utilisant `impl Lifeform` comme type de retour, la fonction `return_lifeform` spécifie que la valeur retournée par la fonction doit nécessairement implémenter ce trait, mais sans spécifier exactement le type concret.

Cette fonctionnalité est particulièrement utile dans le cas de fonctions qui renvoie des fermetures (*closure*) ou des iterateurs, auquel cas le type concret n'est pas forcément exprimable. En effet, les fermetures et les itérateurs produisent des types connus du compilateur seul et/ou très longs à specifier.

Une contrainte importante est que avec cette syntaxe, la fonction toujours cependant renvoyer le même type concret. Dans notre exemple, il n'est pas possible pour la fonction de renvoyer soit `Kelp`, soit `Fish`.
Il faut que cela soit toujours le même type. Le compilateur doit en effet pouvoir connaitre le type concret à la compilation.

In [ ]:
fn adder(value: i32) -> impl Fn(i32) -> i32 {
    move |x| x + value
}
adder(5)(3)

Lorsque les contraintes deviennent complexes, il est possible de les spécifier dans une clause `where` à la fin de la signature de la fonction :

In [ ]:
use std::fmt::Display;
use std::fmt::Debug;

fn some_function<T, U>(t: T, u: U) -> i32
    where T: Display + Clone,     // <-- clause where
          U: Clone + Debug
{ 0 }

Utiliser les bornes pour implémenter de manière conditionnelle
----

Comme déjà vu un peu plus haut, il est possible de spécifier des contraintes sur les paramètres de type des blocks `impl`.
Ce mécanisme peut être utilisé pour implémenter de manière conditionnelle des méthodes en fonction des contraintes sur ces paramètres de type.

In [ ]:
trait Lifeform {
    fn is_alive(&self) -> bool;
}

struct Couple<T> {
    male: T,
    female: T,
}

impl<T> Couple<T> {
    fn new(male: T, female: T) -> Self {
        Self {male, female}
    }
}

impl<T: Lifeform + Default> Couple<T> {
    fn mate(&self) -> Option<T> {
        if self.male.is_alive() && self.female.is_alive() {
            Some(T::default())
        } else {
            None
        }
    }
}

Exercises
===

**Exercice 1:** Simplifiez le code suivant en utilisant de la généricité

In [2]:
trait input {
    fn largest_idiomatic(&self) -> xxx;
}



impl<T> input<T> for  {
fn largest_idiomatic(list: &[i32]) -> i32 {  // trouve le plus grand i32 dans la slice
    let mut largest = list[0];

    for &item in list.iter() {
        if item > largest {
            largest = item;
        }
    }

    largest
}
//
fn largest_i32(list: &[i32]) -> i32 {  // trouve le plus grand i32 dans la slice
    let mut largest = list[0];

    for &item in list.iter() {
        if item > largest {
            largest = item;
        }
    }

    largest
}

fn largest_char(list: &[char]) -> char { // trouve le plus grand char dans la slice
    let mut largest = list[0];

    for &item in list.iter() {
        if item > largest {
            largest = item;
        }
    }

    largest
}

fn main() {
    let number_list = vec![34, 50, 25, 100, 65];

    let result = largest_idiomatic(&number_list);
    println!("The largest number is {}", result);

    let char_list = vec!['y', 'm', 'a', 'q'];

    let result = largest_idiomatic(&char_list);
    println!("The largest char is {}", result);
}
main();

/*
fn main() {
    let number_list = vec![34, 50, 25, 100, 65];    
    
    let result = largest_i32(&number_list);
    println!("The largest number is {}", result);

    let char_list = vec!['y', 'm', 'a', 'q'];

    let result = largest_char(&char_list);
    println!("The largest char is {}", result);
}
main();

The largest number is 100
The largest char is y


> **Astuce:** : Utilisez les traits [`std::cmp::Ord`](https://doc.rust-lang.org/std/cmp/trait.Ord.html) et [`std::marker::Copy`](https://doc.rust-lang.org/std/marker/trait.Copy.html).

**Solution :**

In [ ]:
fn largest<T: PartialOrd + Copy>(list: &[T]) -> T {
    let mut largest = list[0];

    for &item in list.iter() {
        if item > largest {
            largest = item;
        }
    }

    largest
}

fn main() {
    let number_list = vec![34, 50, 25, 100, 65];

    let result = largest_idiomatic(&number_list);
    println!("The largest number is {}", result);

    let char_list = vec!['y', 'm', 'a', 'q'];

    let result = largest_idiomatic(&char_list);
    println!("The largest char is {}", result);
}
main();